In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import tensorflow as tf
import pmaw
import newsapi
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from newsapi import NewsApiClient
from dotenv import load_dotenv

from libs.prep_methods import articles_pull, subreddit_pull, keyword_filter, articles_vader_analyzer, reddit_vader_analyzer, daily_mean


In [2]:
load_dotenv()

True

In [3]:
newsapi_key = os.getenv('NEWSAPI_KEY')

In [4]:
#apple_articles = articles_pull('AAPL OR Apple OR apple')
#apple_articles.to_csv('./Data/Cleaned_Data/apple_articles.csv')

In [5]:
#apple_articles = articles_pull('AAPL OR Apple OR apple')
#apple_articles.to_csv('./Data/Cleaned_Data/apple_articles.csv')

In [6]:
#apple_articles = pd.read_csv('./Data/Cleaned_Data/apple_articles.csv', parse_dates = True, infer_datetime_format = True)

In [7]:
#apple_articles_sentiment = daily_mean('articles', articles_vader_analyzer(apple_articles))

In [8]:
#after = int(dt.datetime(2012, 1, 1, 0, 0).timestamp())
#before = int(dt.datetime(2022, 6, 1, 0, 0).timestamp())

#stockmarket_comments = subreddit_pull('stockmarket', limit = 10000, after = after, before = before)
#stockmarket_comments.to_csv('./Data/Cleaned_Data/stockmarket_comments.csv')

#securityanalysis_comments = subreddit_pull('securityanalysis', limit = 2000000, after = after, before = before)
#securityanalysis_comments.to_csv('../../sandbox/securityanalysis_comments_large.csv')

#algotrading_comments = subreddit_pull('algotrading', limit = 10000, after = after, before = before)
#algotrading_comments.to_csv('./Data/Cleaned_Data/algotrading_comments.csv')

#wallstreetbets_comments = subreddit_pull('wallstreetbets', limit = 2000000, after = after, before = before)
#wallstreetbets_comments.to_csv('.../../sandbox/wallstreetbets_comments_large.csv')

In [9]:
stockmarket_comments = pd.read_csv('../../sandbox/stockmarket_comments_large.csv', lineterminator = '\n', parse_dates = True, infer_datetime_format = True)
#securityanalysis_comments = pd.read_csv('../../sandbox/securityanalysis_comments_large.csv', lineterminator = '\n', parse_dates = True, infer_datetime_format = True)
#algotrading_comments = pd.read_csv('./Data/Cleaned_Data/algotrading_comments.csv')
#wallstreetbets_comments = pd.read_csv('./Data/Cleaned_Data/wallstreetbets_comments.csv')

In [10]:
stockmarket_airbnb_comments = keyword_filter(stockmarket_comments.fillna(''), ['ABNB', 'AirBNB', 'airbnb', 'Airbnb'])
stockmarket_uber_comments = keyword_filter(stockmarket_comments, ['UBER', 'Uber'])

In [27]:
stockmarket_airbnb_daily_sentiment = daily_mean(reddit_vader_analyzer('stockmarket', stockmarket_airbnb_comments))
stockmarket_airbnb_daily_sentiment['datetime'] = pd.to_datetime(stockmarket_airbnb_daily_sentiment['datetime'], infer_datetime_format = True, errors = 'coerce')
stockmarket_airbnb_daily_sentiment = stockmarket_airbnb_daily_sentiment.set_index('datetime')
stockmarket_airbnb_daily_sentiment.index = stockmarket_airbnb_daily_sentiment.index.date

stockmarket_uber_daily_sentiment = daily_mean(reddit_vader_analyzer('stockmarket', stockmarket_uber_comments))
stockmarket_uber_daily_sentiment['datetime'] = pd.to_datetime(stockmarket_uber_daily_sentiment['datetime'], infer_datetime_format = True, errors = 'coerce')
stockmarket_uber_daily_sentiment = stockmarket_uber_daily_sentiment.set_index('datetime')
stockmarket_uber_daily_sentiment.index = stockmarket_uber_daily_sentiment.index.date

/Users/Alexander/Programming/Penn_Fintech/P2G2-ProjectGit/Notebooks/libs/prep_methods.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{subreddit}_compound_sentiment'] = [analyzer.polarity_scores(x)['compound'] for x in df['text']]
/Users/Alexander/Programming/Penn_Fintech/P2G2-ProjectGit/Notebooks/libs/prep_methods.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{subreddit}_positive_sentiment'] = [analyzer.polarity_scores(x)['pos'] for x in df['text']]
/Users/Alexander/Programming/Penn_F

In [28]:
stock_data = pd.read_csv('./Data/Cleaned_Data/stock_data.csv', parse_dates = True, infer_datetime_format = True)
stock_data['date'] = pd.to_datetime(stock_data['date'], infer_datetime_format = True, errors = 'coerce')
stock_data = stock_data.set_index('date')
stock_data.index.name = None

In [29]:
airbnb_prices_volume = stock_data[stock_data['ticker'] == 'ABNB'].drop(columns = 'ticker')
uber_prices_volume = stock_data[stock_data['ticker'] == 'UBER'].drop(columns = 'ticker')

In [36]:
airbnb = pd.concat([stockmarket_airbnb_daily_sentiment, airbnb_prices_volume], axis = 1)
airbnb.ffill(axis = 0, inplace = True)
airbnb.dropna(inplace = True)
airbnb.to_csv('./Data/Cleaned_Data/airbnb.csv')

uber = pd.concat([stockmarket_uber_daily_sentiment, uber_prices_volume], axis = 1)
uber.ffill(axis = 0, inplace = True)
airbnb.dropna(inplace = True)
airbnb.to_csv('./Data/Cleaned_Data/uber.csv')

/usr/local/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/pandas/core/indexes/base.py:345: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  joined_ndarray, lidx, ridx = libjoin.outer_join_indexer(sv, ov)
/usr/local/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/pandas/core/indexes/base.py:3512: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  indexer = self._engine.get_indexer(target._get_engine_target())


In [37]:
airbnb

,stockmarket_compound_sentiment,stockmarket_positive_sentiment,stockmarket_neutral_sentiment,stockmarket_negative_sentiment,close,volume
2020-12-10,0.251840,0.086190,0.881024,0.03281,144.71,70470750.0
2020-12-11,0.634537,0.136875,0.840375,0.02275,139.25,26981241.0
2020-12-12,0.025800,0.151000,0.708500,0.14050,139.25,26981241.0
2020-12-13,0.025800,0.151000,0.708500,0.14050,139.25,26981241.0
2020-12-14,-0.401900,0.000000,0.903000,0.09700,130.00,16966090.0
...,...,...,...,...,...,...
2022-05-25,0.440400,0.081000,0.863000,0.05600,110.40,6170197.0
2022-05-26 00:00:00,0.440400,0.081000,0.863000,0.05600,114.30,6884206.0
2022-05-27 00:00:00,0.440400,0.081000,0.863000,0.05600,120.50,7375113.0
2022-05-31 00:00:00,0.440400,0.081000,0.863000,0.05600,120.87,9089352.0
